In [2]:
import pandas as pd
# Model to apply
import xgboost as xgb # need to set manually what variables are categorical

apl = pd.read_csv('../data/application_record.csv')

In [3]:
apl.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [4]:
apl.shape

(438557, 18)

In [5]:
cred = pd.read_csv('../data/credit_record.csv')

In [6]:
target_list = [1 if i in ['1','2','3','4','5'] else 0 for i in cred['STATUS'] ]

In [7]:
cred['TARGET'] = target_list

In [8]:
cred

,ID,MONTHS_BALANCE,STATUS,TARGET
0,5001711,0,X,0
1,5001711,-1,0,0
2,5001711,-2,0,0
3,5001711,-3,0,0
4,5001712,0,C,0
...,...,...,...,...
1048570,5150487,-25,C,0
1048571,5150487,-26,C,0
1048572,5150487,-27,C,0
1048573,5150487,-28,C,0


In [9]:
cred = cred.groupby('ID').sum()

In [10]:
target_list = [1 if i > 1 else 0 for i in cred['TARGET'] ]

In [11]:
cred['TARGET'] = target_list

In [12]:
df = apl.merge(cred, how='left', on = 'ID')

In [13]:
df.DAYS_EMPLOYED.value_counts()

 365243    75329
-218         262
-1678        251
-249         241
-978         240
           ...  
-13529         1
-10223         1
-8917          1
-11429         1
-8105          1
Name: DAYS_EMPLOYED, Length: 9406, dtype: int64

## Cleaning Data

In [14]:
df = df.drop(['MONTHS_BALANCE','FLAG_MOBIL','DAYS_EMPLOYED'], axis = 1)

In [15]:
df = df[pd.notnull(df['TARGET'])]

In [16]:
df.TARGET.value_counts()

0.0    34358
1.0     2099
Name: TARGET, dtype: int64

In [17]:
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TARGET
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,1,0,0,NaN,2.0,0.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,1,0,0,NaN,2.0,0.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,0,0,0,Security staff,2.0,0.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,0,1,1,Sales staff,1.0,0.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,0,1,1,Sales staff,1.0,0.0


In [18]:
df_aux = pd.get_dummies(df['CODE_GENDER'],drop_first=True,prefix='CODE_GENDER')

In [19]:
df_aux2 = pd.get_dummies(df['FLAG_OWN_CAR'],drop_first=True,prefix='FLAG_OWN_CAR')

In [20]:
df_aux3 = pd.get_dummies(df['FLAG_OWN_REALTY'],drop_first=True,prefix='FLAG_OWN_REALTY')

In [21]:
df = pd.concat([df.drop('CODE_GENDER', axis=1), df_aux], axis=1);

In [22]:
df = pd.concat([df.drop('FLAG_OWN_CAR', axis=1), df_aux2], axis=1);

In [23]:
df = pd.concat([df.drop('FLAG_OWN_REALTY', axis=1), df_aux3], axis=1);

In [24]:
import numpy as np
cat_vars = df.select_dtypes(exclude=[np.number])
cat_vars.describe()

,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE
count,36457,36457,36457,36457,25134
unique,5,5,5,6,18
top,Working,Secondary / secondary special,Married,House / apartment,Laborers
freq,18819,24777,25048,32548,6211


In [25]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
for c in cat_vars.columns:
    lbl = LabelEncoder() 
    lbl.fit(list(df[c].values)) 
    df[c] = lbl.transform(list(df[c].values))

In [26]:
from datetime import date, timedelta,datetime

In [27]:
#df['date'] = pd.to_datetime('today')

In [28]:
df.head()

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TARGET,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y
0,5008804,0,427500.0,4,1,0,4,-12005,1,0,0,18,2.0,0.0,1,1,1
1,5008805,0,427500.0,4,1,0,4,-12005,1,0,0,18,2.0,0.0,1,1,1
2,5008806,0,112500.0,4,4,1,1,-21474,0,0,0,16,2.0,0.0,1,1,1
3,5008808,0,270000.0,0,4,3,1,-19110,0,1,1,14,1.0,0.0,0,0,1
4,5008809,0,270000.0,0,4,3,1,-19110,0,1,1,14,1.0,0.0,0,0,1


In [29]:
#df['time_since'] = pd.to_timedelta(df['DAYS_BIRTH'],'d')
#df['days_birth'] = df['date'] + df['time_since']


In [30]:
df['DAYS_BIRTH'] = round(df['DAYS_BIRTH']/-365,0)

In [31]:
#df['DAYS_EMPLOYED'] = round(df['DAYS_EMPLOYED']/-365,0)

In [32]:
df = df.rename(columns={"DAYS_BIRTH": "AGE", "DAYS_EMPLOYED": "YEAR_EMPLOYED"})

In [33]:
df.AGE = df.AGE.astype(int)

In [34]:
#df.YEAR_EMPLOYED = df.YEAR_EMPLOYED.astype(int)

In [35]:
df.TARGET = df.TARGET.astype(int)

In [36]:
df.CNT_FAM_MEMBERS = df.CNT_FAM_MEMBERS.astype(int)

In [37]:
df.head()

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,AGE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TARGET,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y
0,5008804,0,427500.0,4,1,0,4,33,1,0,0,18,2,0,1,1,1
1,5008805,0,427500.0,4,1,0,4,33,1,0,0,18,2,0,1,1,1
2,5008806,0,112500.0,4,4,1,1,59,0,0,0,16,2,0,1,1,1
3,5008808,0,270000.0,0,4,3,1,52,0,1,1,14,1,0,0,0,1
4,5008809,0,270000.0,0,4,3,1,52,0,1,1,14,1,0,0,0,1


In [38]:
#df.YEAR_EMPLOYED.value_counts()

In [39]:
df.to_csv('target_con_edat.csv')

## Preprocessing

In [40]:
df.dtypes

ID                       int64
CNT_CHILDREN             int64
AMT_INCOME_TOTAL       float64
NAME_INCOME_TYPE         int64
NAME_EDUCATION_TYPE      int64
NAME_FAMILY_STATUS       int64
NAME_HOUSING_TYPE        int64
AGE                      int64
FLAG_WORK_PHONE          int64
FLAG_PHONE               int64
FLAG_EMAIL               int64
OCCUPATION_TYPE          int64
CNT_FAM_MEMBERS          int64
TARGET                   int64
CODE_GENDER_M            uint8
FLAG_OWN_CAR_Y           uint8
FLAG_OWN_REALTY_Y        uint8
dtype: object

In [41]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
splitter = train_test_split
"-----------------------"

df_train, df_test = splitter(df, test_size = 0.2, random_state = 42)
print("Dataset shape: {shape}".format(shape = df_train.shape))
print("Dataset shape: {shape}".format(shape = df_test.shape))


Dataset shape: (29165, 17)
Dataset shape: (7292, 17)


In [42]:
def get_specific_columns(df, data_types, to_ignore = list(), ignore_target = False):
    columns = df.select_dtypes(include=data_types).columns
    if ignore_target:
        columns = filter(lambda x: x not in to_ignore, list(columns))
    return list(columns)

In [43]:
target = "TARGET"
variables = list(get_specific_columns(df, ["float64", "int64","uint8"], [target], ignore_target = True))

X_train= df_train[variables]
y_train = df_train[target]

X_test= df_test[variables]
y_test = df_test[target]


In [44]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


X_new = SelectKBest(chi2, k=3).fit_transform(X_train, y_train)

In [45]:
X_new

array([[5.037048e+06, 1.350000e+05, 4.500000e+01],
       [5.044630e+06, 1.350000e+05, 2.800000e+01],
       [5.079079e+06, 1.800000e+05, 3.500000e+01],
       ...,
       [5.047710e+06, 7.650000e+04, 4.900000e+01],
       [5.009886e+06, 1.575000e+05, 5.900000e+01],
       [5.062632e+06, 5.850000e+05, 5.200000e+01]])

In [46]:
# from sklearn import svm
# from sklearn.model_selection import cross_val_score
# clf = svm.SVC(kernel='linear', C=1, random_state=42)
# scores = cross_val_score(clf, X_train, y_train, cv=5)
%store X_train
%store y_train
%store X_test
%store y_test
%store df

Stored 'X_train' (DataFrame)
Stored 'y_train' (Series)
Stored 'X_test' (DataFrame)
Stored 'y_test' (Series)
Stored 'df' (DataFrame)
